<a href="https://colab.research.google.com/github/shaem454/Coursera_Capstone/blob/master/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [158]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import sys, math
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium # map rendering library
!pip install foursquare
!pip install git+https://github.com/dacog/foursquare_api_tools.git#egg=foursquare_api_tools
import foursquare as fs
from foursquare_api_tools import foursquare_api_tools as ft
print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


Define Client credentials

In [159]:
CLIENT_ID = '2WEIVPXNKBQZB1W2O220DXKIIMIMVOJQLG2NXSZZYO4KJI3L' # your Foursquare ID
CLIENT_SECRET = 'ZIZYUWH5IUNFJ3MVBIUESNHJ0BSO5MX3MPTCYF4WYGMHAWVT' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: 2WEIVPXNKBQZB1W2O220DXKIIMIMVOJQLG2NXSZZYO4KJI3L


Define the corresponding URL, send the GET Request to Foursquare API and examine the results

In [0]:

LIMIT = 500 # Maximum is 100
radius = 1500
cities = ['Brunswick VIC', 'Auckland', 'Wellington', 'Christchurch', 'Hamilton NZ']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&radius={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        radius,
        "4bf58dd8d48988d1d2941735") # sushi restaurant id
    results[city] = requests.get(url).json()

Create a dataframe for each cities' sushi venues' name, address, latitude and longitude.

In [0]:
sushi_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    sushi_venues[city] = venues[['venue.name', 'venue.location.lat', 'venue.location.lng']]
    sushi_venues[city].columns = ['Name', 'Lat', 'Lng']

In [162]:
print(sushi_venues[cities[0]])

                                Name        Lat         Lng
0            Kumo Izakaya & Sake Bar -37.773424  144.971492
1                     Shinwa Izakaya -37.774137  144.960878
2                          Matsumoto -37.777235  144.970946
3             Hinoki Japanese Pantry -37.800580  144.983955
4                        Sushi Sushi -37.775609  144.961931
5               Ichi Ni Nana Izakaya -37.803757  144.977165
6             JQ Sushi & Asian Taste -37.772210  144.949650
7                    Oshin Sushi Bar -37.784245  144.959137
8             MoMo Sushi (Pelham St) -37.802321  144.961089
9                           Sea Salt -37.798332  144.967386
10                        MoMo Sushi -37.801157  144.963878
11  Star Sushi Bar and Hand Car Wash -37.741020  144.964870
12              Sumos Noodle & Sushi -37.788770  144.989640
13                         Sushi Ten -37.795136  144.971074
14               Yes Sushi Yes Juice -37.796630  144.984518
15                        Nori Sushi -37

Create a Folium map for each city, with markers added for each sushi venue within the defined 1.5 km radius.
Determine how many sushi venues are found within this radius.

In [163]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=14)

    # add markers to map
    for lat, lng, label in zip(sushi_venues[city]['Lat'], sushi_venues[city]['Lng'], sushi_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
    print(f"Total number of sushi places in {city} within {radius}m of City Centre= ", results[city]['response']['totalResults'])


Total number of sushi places in Brunswick VIC within 1500m of City Centre=  54
Total number of sushi places in Auckland within 1500m of City Centre=  46
Total number of sushi places in Wellington within 1500m of City Centre=  35
Total number of sushi places in Christchurch within 1500m of City Centre=  8
Total number of sushi places in Hamilton NZ within 1500m of City Centre=  18


Visually inspect the map of each city

In [164]:
print(cities[0])
maps[cities[0]]

Brunswick VIC


In [165]:
print(cities[1])
maps[cities[1]]

Auckland


In [166]:
print(cities[2])
maps[cities[2]]

Wellington


In [167]:
print(cities[3])
maps[cities[3]]

Christchurch


In [168]:
print(cities[4])
maps[cities[4]]

Hamilton NZ


To determine the best location to find a house/apartment which would limit the mean distance to each sushi venue, find the mean coordinate from each cities' sushi venues' latitudes and longitudes.

In [0]:
for city in cities:
  venues_mean_coor = [sushi_venues[city]['Lat'].mean(), sushi_venues[city]['Lng'].mean()]

In [170]:
 for city in cities:
    print(city)
    print("Mean Coordinate")
    print([sushi_venues[city]['Lat'].mean(), sushi_venues[city]['Lng'].mean()])

Brunswick VIC
Mean Coordinate
[-37.789369092799696, 144.96825665600176]
Auckland
Mean Coordinate
[-36.85871458790793, 174.76689561211802]
Wellington
Mean Coordinate
[-41.287690684226455, 174.77693456845236]
Christchurch
Mean Coordinate
[-43.535147979870764, 172.63369264456878]
Hamilton NZ
Mean Coordinate
[-37.785967445945424, 175.2800381542179]


Add the mean location to each map, with lines running from the mean location to each venue so that it can be visualised easily.

In [0]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=14)
    venues_mean_coor = [sushi_venues[city]['Lat'].mean(), sushi_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(sushi_venues[city]['Lat'], sushi_venues[city]['Lng'], sushi_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])


Visualise the maps with the mean coordinate and lines to each venue

In [172]:
print(cities[0])
maps[cities[0]]

Brunswick VIC


In [173]:
print(cities[1])
maps[cities[1]]

Auckland


In [174]:
print(cities[2])
maps[cities[2]]

Wellington


In [175]:
print(cities[3])
maps[cities[3]]

Christchurch


In [176]:
print(cities[4])
maps[cities[4]]

Hamilton NZ
